In [43]:
import torch
import torch.nn as nn

In [44]:
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms

In [45]:
# 定义超参数 
input_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3  #训练的总循环周期
batch_size = 64  #一个撮（批次）的大小，64张图片

# 训练集
train_dataset = datasets.MNIST(root='./data',  
                            train=True,   
                            transform=transforms.ToTensor(),  
                            download=True) 

# 测试集
test_dataset = datasets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

d:\Anaconda1\envs\pytorch\lib\site-packages\torchvision\datasets\mnist.py:119: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(os.path.join(self.processed_fo

In [48]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1=nn.Sequential(
            nn.Conv2d(
                in_channels=1,
                out_channels=16,
                kernel_size=5,
                stride=1, 
                padding=2,
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )

        self.conv2=nn.Sequential(
                nn.Conv2d(16,32,5,1,2),
                nn.ReLU(),
                nn.Conv2d(32,32,5,1,2),
                nn.ReLU(),
                nn.MaxPool2d(2),
            
        )

        self.conv3=nn.Sequential(
            nn.Conv2d(32,64,5,1,2),
            nn.ReLU(),
        )

        self.out=nn.Linear(64*7*7,10) # 全连接层得到的结果



    def forward(self,x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=x.view(x.size(0),-1)  # 展平 结果为[batch_size,64*7*7]
        output=self.out(x)
        return output

In [134]:
def accuracy(predictions,labels):
    pred=torch.max(predictions.data,1)[1]
    rights=pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [143]:
net=CNN()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.001)
for epoch in range(num_epochs):
    train_rights=[]
    for batch_idx,(data,target) in enumerate(train_loader):
        # print(data,target)
        net.train()
        output=net(data)
        optimizer.zero_grad()
        loss=criterion(output,target)
        loss.backward()
        optimizer.step()
        right=accuracy(output,target)
        train_rights.append(right)
    
        # pred=torch.max(output.data,1)[1]
        # print(torch.max(output.data,1)[1])
        # print(pred.eq(target.data.view_as(pred)).sum())
        # print(target.data.size())
        # print(output.data.size())
    
    
        if batch_idx%100==0:
            net.eval()
            val_rights=[]
    
            for (data,target) in test_loader:
                output=net(data)
                right=accuracy(output,target)
                val_rights.append(right)
            
            train_r=(sum([tup[0] for tup in train_rights]),sum([tup[1] for tup in train_rights]))
            val_r=(sum([tup[0] for tup in val_rights]),sum([tup[1] for tup in val_rights]))
    
            print("当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t\
                  训练集准确率: {:.2f}%\t测试集准确率: {:.2f}%"
                  .format(epoch,batch_idx*batch_size,len(train_loader.dataset),
                  100.*batch_idx/len(train_loader),loss.item(),
    
                  100.*train_r[0]/train_r[1],100.*val_r[0]/val_r[1] 
                )
            )
    # print(train_rights)

当前epoch: 0 [0/60000 (0%)]	损失: 2.302417	                  训练集准确率: 9.38%	测试集准确率: 23.00%
当前epoch: 0 [6400/60000 (11%)]	损失: 0.138415	                  训练集准确率: 77.78%	测试集准确率: 94.51%
当前epoch: 0 [12800/60000 (21%)]	损失: 0.040330	                  训练集准确率: 86.35%	测试集准确率: 96.25%
当前epoch: 0 [19200/60000 (32%)]	损失: 0.031716	                  训练集准确率: 89.64%	测试集准确率: 96.56%
当前epoch: 0 [25600/60000 (43%)]	损失: 0.071266	                  训练集准确率: 91.56%	测试集准确率: 97.54%
当前epoch: 0 [32000/60000 (53%)]	损失: 0.092381	                  训练集准确率: 92.71%	测试集准确率: 97.56%
当前epoch: 0 [38400/60000 (64%)]	损失: 0.064383	                  训练集准确率: 93.53%	测试集准确率: 98.08%
当前epoch: 0 [44800/60000 (75%)]	损失: 0.039310	                  训练集准确率: 94.19%	测试集准确率: 98.10%
当前epoch: 0 [51200/60000 (85%)]	损失: 0.020843	                  训练集准确率: 94.66%	测试集准确率: 98.51%
当前epoch: 0 [57600/60000 (96%)]	损失: 0.036520	                  训练集准确率: 95.06%	测试集准确率: 98.34%
当前epoch: 1 [0/60000 (0%)]	损失: 0.031024	                  训练集准确率: 98.44%	测试集准确率: 98.49%
